In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import pyodbc
import datetime as dt
import re
import gc

In [2]:
df_id_campanhas = pd.read_csv(r'Q:\\Drives compartilhados\\CRM - STZ\\CRM\\_Carga de Arquivos\\Querys\\ID Campanhas BTG.csv', sep=';')

with open(r'C:\Users\roberson.valle\Meu Drive\Github\API_ALLIN.json') as arquivo:
    payload = json.load(arquivo)

In [3]:
url_post = "http://transacional-apiv2.allin.com.br/oauth/token"

response_post = requests.post(url_post, data=payload)

In [4]:
url_get = "http://transacional-apiv2.allin.com.br/api/report/?dt_envio="
data_get = '2022-03-16'
page = '1'
json_get = []
headers = {
  'Authorization': f'Bearer {response_post.json()["access_token"]}'
}

qtd_paginas = requests.get((f'{url_get}{data_get}&page={page}'), headers=headers).json()['last_page']
#qtd_paginas = 1
for i in tqdm(range(qtd_paginas)):
    response_get = requests.get((f'{url_get}{data_get}&page={(i+1)}'), headers=headers).json()
    json_get.append(response_get)

100%|██████████| 58/58 [02:13<00:00,  2.31s/it]


In [5]:
df = pd.DataFrame(json_get[0]['data'])

for i in range((qtd_paginas-1)):
    df_aux = pd.DataFrame(json_get[i+1]['data'])
    df = pd.concat([df, df_aux], ignore_index=True)

df = df.query('fl_envio==True and dt_enviado!=None')

In [6]:
df_completo = pd.merge(df,df_id_campanhas,left_on='nm_envio',right_on='nome_regra',how='left')
df_completo = df_completo[df_completo['dt_abertura'].notnull()]

In [7]:
server = 'PASSODETORRES' 
database = 'CRM_DATALAKE' 
conexao = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database)
cursor = conexao.cursor()

In [8]:
linhas_com_erro = 0

try:
    for index, row in tqdm(df_completo.iterrows()):
        try:
            cursor.execute(f"""
            INSERT INTO [CRM_DATALAKE].[CRM].[ABERTOS_STZ]
                ([EMAIL]
                ,[ID_CAMPANHA]
                ,[DATA_ENVIO]
                ,[DATA_ABERTURA]
                ,[DATA_ATUALIZACAO]
                ,[TIPO_COMUNICACAO])
            VALUES(
                CASE WHEN '{row.nm_email}' = 'nan' THEN NULL ELSE '{row.nm_email}' END
                ,'{str(pd.to_datetime(row.dt_enviado, errors='ignore')).replace('-','')[:8]}BTG{str(int(row.campaign_id)).rjust(2,'0')}'
                ,cast('{str(pd.to_datetime(row.dt_enviado))}' as date)
                ,CASE WHEN '{str(pd.to_datetime(row.dt_abertura))}' = 'none' THEN NULL ELSE CAST('{str(pd.to_datetime(row.dt_abertura))}' as date) END
                ,GETDATE()
                ,'BTG')
            """)
        except:
            linhas_com_erro+=1
    try:
        print(f'Total de linhas: {len(df_completo)}')
        print(f'Linhas com erros: {linhas_com_erro}')
        if linhas_com_erro == 0:
            print('COMMIT')
            conexao.commit()
        else:
            print('ROLLBACK')
            conexao.rollback()    
    except:
        conexao.rollback()
        print('Erro no commit!')
except:
    conexao.rollback()
    print('Erro na Carga')

cursor.close()

4360it [00:39, 109.50it/s]

Total de linhas: 4360
Linhas com erros: 0
COMMIT


In [9]:
df_completo.dt_envio.unique()[0]

'2022-03-16'